## Building simple graph using Nobel_prize JSON data
#### we just use simple edges as connections, and graph is formed of a set of cliques based on the records

In [4]:
import json
import pandas as pd
import networkx as nx
from graphgen import create_graph
from pprint import pprint

file1 = '../data/Nobel_prize.json'
file2 = '../data/Nobel_laureate.json'
file3 = '../data/Nobel_country.json' # this file has simple json structure

In [5]:
with open(file2) as json_file:
    json_data2 = json.load(json_file)

In [6]:
json_data2['laureates'][100]

{'id': '103',
 'firstname': 'Ben Roy',
 'surname': 'Mottelson',
 'born': '1926-07-09',
 'died': '0000-00-00',
 'bornCountry': 'USA',
 'bornCountryCode': 'US',
 'bornCity': 'Chicago, IL',
 'gender': 'male',
 'prizes': [{'year': '1975',
   'category': 'physics',
   'share': '3',
   'motivation': '"for the discovery of the connection between collective motion and particle motion in atomic nuclei and the development of the theory of the structure of the atomic nucleus based on this connection"',
   'affiliations': [{'name': 'Nordita',
     'city': 'Copenhagen',
     'country': 'Denmark'}]}]}

In [ ]:
nodes_mapper = {
    'nodes': [
        {
            'type': 'Affiliations',
            'path': '/prizes/affiliations',
            'key' : [
                {'name': 'name', 'raw': '/prizes/affiliations/name'}
            ],
        },
        {
            'type': 'Prize',
            'path': '/prizes',
            'key' : [
                {'name': 'category', 'raw': '/prizes/category'}
            ],
            'attributes': [
                {'name': 'category', 'raw': '/prizes/category'},
            ]
        }
    ]
}
# this model doesn't have any specific edge, just connect all nodes in each record (clique)
clique_mapper = {
    'cliques': [
        {
            'type': '__default__',
            'nodes': [
                {
                    'type': 'Affiliations',
                    'path': '/prizes/affiliations',
                    'key' : [
                        {'name': 'name', 'raw': '/prizes/affiliations/name'}
                    ]
                },
                {
                    'type': 'Prize',
                    'path': '/prizes',
                    'key' : [
                        {'name': 'category', 'raw': '/prizes/category'}
                    ]
                },
            ]
        }
    ]
}

In [ ]:
g = nx.MultiGraph()

g = create_graph(g, graph_mapper = nodes_mapper, 
                 data_provider = json_data2['laureates'])

g = create_graph(g, graph_mapper = clique_mapper, 
                 data_provider = json_data2['laureates'])

In [ ]:
type(g)

In [ ]:
nx.number_of_nodes(g)

In [ ]:
nx.number_of_edges(g)

In [ ]:
pprint(json_data2['laureates'][216])

In [ ]:
print(g.node['Affiliations_University of Cambridge'])

In [ ]:
print(g.node['Prize_chemistry'])
print(g.node['Prize_physics'])

In [ ]:
pprint(g.get_edge_data('Affiliations_University of Cambridge', 'Prize_chemistry'))

In [ ]:
pprint(g.get_edge_data('Prize_chemistry', 'Affiliations_University of Cambridge'))